# This version uses the same logic as in v_10 but uses one week as its timeframe

#### Have also updated the FCR and aFRR functions to make them faster and make FCR handle missing values

In [1]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, analysis, timeframes, data_handling
import numpy as np
import pickle
from datetime import datetime



In [ ]:
timeframe = timeframes.one_day
L, M, F, H, freq_data, power_meter_dict, consumption_data = utils.get_all_sets(timeframe)

In [ ]:
L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m = utils.get_parameters(L,M,H)

In [ ]:

markets_name_dict = {market.name: market for market in M}
market_names = list(markets_name_dict.keys())
price_list = [market.price_data for market in M]
volume_list = [market.volume_data for market in M]


In [ ]:
print(f"Amount of markets : {len(M)}")
print(f"Amount of meters : {len(L)}")
print(f"Amount of meters with direction up or both : {len(L_u)}")
print(f"Amount of meters with direction down or both : {len(L_d)}")
print(f"Amount of hours : {len(H)}")

In [ ]:
total_up_flex = np.sum(Fu_h_l) # total available flex volume up
total_down_flex = np.sum(Fd_h_l) # total available flex volume down
total_response_time = np.sum(R_h_l) # total response time
#total_flex = total_up_flex + total_down_flex
average_response_time = total_response_time/ (len(H)*len(L))
hourly_flex_up = total_up_flex/len(H)
hourly_flex_down = total_down_flex/len(H)

print(f"Total up flex volume: {total_up_flex} MW")
print(f"Total down flex volume: {total_down_flex} MW")
print(f"Average flex volume pr hour up: {hourly_flex_up} MWh")
print(f"Average flex volume pr hour down: {hourly_flex_down} MWh")
print(f"Average response time: {average_response_time} seconds")

In [ ]:
dominant_directions = [utils.get_dominant_direction(freq_data, hour) for hour in H]

In [ ]:
Ir_hlm, Ia_hlm, Va_hm = utils.get_income_dictionaries(H, M, L, dominant_directions, Fu_h_l, Fd_h_l, P_h_m, Vp_h_m, F, markets_name_dict, timeframe)


In [ ]:
compatible_list = utils.get_compatibility_dict(L, M)

In [ ]:
data_handling.save_collections(L = L, M = M, F = F, H = H, freq_data= freq_data, power_meter_dict = power_meter_dict, consumption_data= consumption_data, L_u = L_u, L_d = L_d, Fu_h_l = Fu_h_l, Fd_h_l = Fd_h_l, R_h_l = R_h_l, P_h_m = P_h_m, Vp_h_m = Vp_h_m, Vm_m = Vm_m, R_m = R_m, Ir_hlm = Ir_hlm, Ia_hlm = Ia_hlm, Va_hm = Va_hm, dominant_directions = dominant_directions, compatible_list = compatible_list, pkl_filename= "one_day_collections.pkl")

In [ ]:
test_model, x, y, w, d  = utils.run_optimization_model(L, M, H, F, Ir_hlm, Ia_hlm, Va_hm, Vp_h_m, Vm_m, R_m, R_h_l, Fu_h_l, Fd_h_l, dominant_directions, compatible_list, "test_model.log", "one_day_model")

In [ ]:
def test_solution_validity(x, y, w, Va_hm, L, M, H, dominant_directions, F):
    """ function to test the validity of the solution provided by a solver

    Args:
        x (dict): dictionary of the binary variable which tells if an asset is connected to a market
        y (dict): dictionary of the binary variable which tells if a market has any bids
        w (dict): dictionary of the binary variable which tells if a market is activated
        L (list(PowerMeter)): list of powermeter objects with the data for each meter within the timeframe
        M (list(ReserveMarket)): list of reservemarket objects with the data for each market within the timeframe
        H (list(pd.TimeStamp)): list of hourly timestamps within the timeframe
        dominant_directions (list(str)): list of the dominant direction for each hour
        F (pd.DataFrame): dictionary for frequency data
    Returns:
        str : a string that tells if the solution is valid. If not valid, the function will raise an error
    """
    for h, hour in enumerate(H):
        for l, load in enumerate(L):
            # Each asset can only be connected to one market at a time
            assert round(sum(x[h, l, m].X for m in range(len(M))), 5) <= 1, f"Asset {l} connected to multiple markets at hour {h}"
            for m, market in enumerate(M):
                x_val= round(x[h, l, m].X, 5)
                # Directionality constraints
                if load.direction == "up" and market.direction == "down":
                    assert x_val== 0, f"Up-direction asset {l} connected to down-direction market {m} at hour {h}"
                elif load.direction == "down" and market.direction == "up":
                    assert x_val == 0, f"Down-direction asset {l} connected to up-direction market {m} at hour {h}"
                #elif market.direction == "both" and load.direction != "both":
                    #assert x[h, l, m].X == 0, f"Asset {l} with specific direction connected to both-direction market {m} at hour {h}"
                elif market.area != load.area:
                    assert x_val == 0, f"Asset {l} in area {load.area} connected to market {m} in area {market.area} at hour {h}"
                
                # Response time constraints
                assert x_val * load.response_time <= market.response_time * round(y[h, m].X, 5), f"Asset {l} connected to market {m} at hour {h} violates response time constraint"
                
        for m, market in enumerate(M):
            # Connect the binary variables by using big M
            assert round(sum(x[h, l, m].X for l in range(len(L))), 5) <= len(L) * round(y[h, m].X, 5), f"More than allowed assets connected to market {m} at hour {h} to market {m}"

            #total_flex_volume = sum(x[h, l, m].X * load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L))

            # Min volume constraint
            if market.direction == "up":
                total_flex_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
            elif market.direction == "down":
                total_flex_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")
            else: # direction = "both"
                if dominant_directions[h] == "up":
                    total_flex_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
                else:
                    total_flex_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")
            
            assert round(total_flex_volume, 5) >= market.min_volume * y[h, m].X, f"Minimum volume constraint violated at hour {h} for market {m}"
            
            # Max volume constraint for both capacity and activation
            if market.direction == "up":
                total_max_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")

            elif market.direction == "down":
                total_max_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")

            else:
                """if dominant_directions[h] == "up":
                    total_max_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
                else:
                    total_max_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")"""
                total_up_max_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
                total_down_max_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")
                up_frac, down_frac = F[h,m]
                total_max_volume = (total_up_max_volume * up_frac + total_down_max_volume * down_frac)
            
             # Assert the constraints
            activation_constraint = round(total_max_volume, 5)  * round(w[h,m].X, 5) <= Va_hm[h,m]
            assert activation_constraint, f"Activation constraint violated for hour {h}, market {m}"
            market_max_volume = market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1]
            assert total_max_volume <= market_max_volume * round(y[h,m].X, 5), f"Maximum volume constraint violated at hour {h} for market {m}"
    return "Solution is valid"

In [ ]:
test_solution_validity(x, y, w, Va_hm, L, M, H, dominant_directions= dominant_directions, F = F)

In [ ]:
sum(x[7, l, 21].X for l in range(len(L)))

In [ ]:
len(L) * y[7, 21].X

In [ ]:
round(sum(x[7, 839, m].X for m in range(len(M))), 6)

In [ ]:
import pickle

In [ ]:
# Extract binary variable values from the original model
new_x_values = {(h, l, m): test_model.getVarByName(f"x_{h}_{l}_{m}").X for h in range(len(H)) for l in range(len(L)) for m in range(len(M))}


In [ ]:
# Load the saved values
with open('current_x_values_for_week.pkl', 'rb') as f:
    original_x_values = pickle.load(f)

old_dict = utils.get_market_count_dict(original_x_values)
new_dict = utils.get_market_count_dict(new_x_values)

differences = {}
for key in old_dict:
    if not old_dict[key].equals(new_dict[key]):
        differences[key] = (new_dict[key], old_dict[key])
        

for key, (orig_val, mod_val) in differences.items():
    print(f"Difference for hour {key}: \n Original={display(orig_val)}, \n  Modified={display(mod_val)}")


In [ ]:
# Extract binary variable values from the original model
current_x_values = {(h, l, m): test_model.getVarByName(f"x_{h}_{l}_{m}").X for h in range(len(H)) for l in range(len(L)) for m in range(len(M))}

# Save these values
with open('current_x_values_for_week.pkl', 'wb') as f:
    pickle.dump(current_x_values, f)
                